## Data Analysis, Exploration, and Feature Engineering
Now that the data is ingested into Athena we can begin to explore the data and perform some feature engineering. Since the data is provided as a Pandas DataFrame we can use familiar Pandas function and data science libraries to explore and prepare the data.

<div style="border: 1px solid black; padding: 10px; background-color: #ffffcc; color: black;">
<strong>Note:</strong> Make sure to fully run the first notebook to ingest the data into Athena before running this notebook.
</div>

In [ ]:
%pip install -Uqq category-encoders
%pip install -Uqq seaborn

In [ ]:
import pandas as pd
import sagemaker
import awswrangler as wr
import seaborn as sns
import numpy as np
import json
from pathlib import Path
import matplotlib.pyplot as plt

wr.engine.set("python")
wr.memory_format.set("pandas")

In [2]:
# load values from the first notebook

if not Path("lab_values.json").exists():
    raise FileNotFoundError("Please run the first notebook first.")
else:
    lab_values = json.loads(Path("lab_values.json").read_text())
    database_name = lab_values["database_name"]
    table_name = lab_values["wrangler_parquet_table_name"]

In [3]:
# read the entire table into a pandas dataframe
# you can also pass in the chunksize parameter to read the table in chunks
df = wr.athena.read_sql_table(table_name, database=database_name)

In [ ]:
# compute the summary statistics
df.describe(percentiles=[0.25, 0.5, 0.75])

Since the data is provided as a Pandas DataFrame we can use common visualization libraries such as Matplotlib and Seaborn to explore the data.

In [5]:
def plot_distribution(df, column_name):
    """
    Plots the distribution of numeric values in a specified column of a DataFrame.
    """
    sns.displot(df[column_name], kde=True, bins=30)
    plt.xlabel(column_name)
    plt.ylabel('Density')
    plt.title(f'Distribution of {column_name}')
    plt.show()

In [ ]:
plot_distribution(df, "ti_ln_remaining_term")

In [ ]:
plot_distribution(df, "ti_cu_cust_type")

In [ ]:
plt.figure(figsize=(12, 6))
sns.violinplot(x="ti_ln_purpose", y="ti_ln_original_loan_amount", data=df)
plt.show()

Data science libraries such as category_encoders and scikit-learn can be used to perform feature engineering and data preprocessing.

In [9]:
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from category_encoders.james_stein import JamesSteinEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion


# treat all numeric columns that have more than 50 unique values as continuous
numeric_features = df[df.nunique()[df.nunique() > 25].index].select_dtypes("number").columns

category_features = [col for col in df.columns if col not in numeric_features and "date" not in col]

date_features = [col for col in df.columns if "date" in col and col not in ["ti_ln_date_open_year", "ti_ln_date_open_month"]]

numeric_preprocessor = ColumnTransformer(transformers = [("scaler", StandardScaler(), numeric_features)], remainder="drop")
category_preprocessor = ColumnTransformer(transformers = [("encoder", JamesSteinEncoder(), category_features)], remainder="drop")


def extract_date_features(df):
    df = df.copy()
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col + '_year'] = df[col].dt.year
            df[col + '_month'] = df[col].dt.month
            df[col + '_day'] = df[col].dt.day
    return df.drop(columns=[col for col in df.columns if pd.api.types.is_datetime64_any_dtype(df[col])])

date_preprocessor = FunctionTransformer(extract_date_features, validate=False)

combined_preprocessor = FeatureUnion(transformer_list=[
    ("numeric", numeric_preprocessor),
    ("categorical", category_preprocessor),
    ("date", date_preprocessor)
])

In [10]:
new_date_features = [f"{col}_{suffix}" for col in date_features for suffix in ['year', 'month', 'day']]
all_columns = numeric_features.tolist() + category_features + new_date_features
transformed_data = combined_preprocessor.fit_transform(df, df["ti_ln_balance"])

np.save("transformed_data", transformed_data)

### Analysis with SQL

We can also use SQL queries to explore the data. This can be useful for large datasets where it is not feasible to load the entire dataset into memory.

In [11]:
profile_numeric_sql = f"""
WITH percentiles AS (
    SELECT
        'ti_ln_remaining_term' AS column_name,
        MIN(ti_ln_remaining_term) AS min_value,
        approx_percentile(ti_ln_remaining_term, 0.25) AS p25,
        approx_percentile(ti_ln_remaining_term, 0.50) AS p50,
        approx_percentile(ti_ln_remaining_term, 0.75) AS p75,
        MAX(ti_ln_remaining_term) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_balance' AS column_name,
        MIN(ti_ln_balance) AS min_value,
        approx_percentile(ti_ln_balance, 0.25) AS p25,
        approx_percentile(ti_ln_balance, 0.50) AS p50,
        approx_percentile(ti_ln_balance, 0.75) AS p75,
        MAX(ti_ln_balance) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_installment_due' AS column_name,
        MIN(ti_ln_installment_due) AS min_value,
        approx_percentile(ti_ln_installment_due, 0.25) AS p25,
        approx_percentile(ti_ln_installment_due, 0.50) AS p50,
        approx_percentile(ti_ln_installment_due, 0.75) AS p75,
        MAX(ti_ln_installment_due) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_val_payments' AS column_name,
        MIN(ti_ln_val_payments) AS min_value,
        approx_percentile(ti_ln_val_payments, 0.25) AS p25,
        approx_percentile(ti_ln_val_payments, 0.50) AS p50,
        approx_percentile(ti_ln_val_payments, 0.75) AS p75,
        MAX(ti_ln_val_payments) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_val_interest' AS column_name,
        MIN(ti_ln_val_interest) AS min_value,
        approx_percentile(ti_ln_val_interest, 0.25) AS p25,
        approx_percentile(ti_ln_val_interest, 0.50) AS p50,
        approx_percentile(ti_ln_val_interest, 0.75) AS p75,
        MAX(ti_ln_val_interest) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_val_total_fees' AS column_name,
        MIN(ti_ln_val_total_fees) AS min_value,
        approx_percentile(ti_ln_val_total_fees, 0.25) AS p25,
        approx_percentile(ti_ln_val_total_fees, 0.50) AS p50,
        approx_percentile(ti_ln_val_total_fees, 0.75) AS p75,
        MAX(ti_ln_val_total_fees) AS max_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_final_charge_cycle' AS column_name,
        MIN(ti_ln_final_charge_cycle) AS min_value,
        approx_percentile(ti_ln_final_charge_cycle, 0.25) AS p25,
        approx_percentile(ti_ln_final_charge_cycle, 0.50) AS p50,
        approx_percentile(ti_ln_final_charge_cycle, 0.75) AS p75,
        MAX(ti_ln_final_charge_cycle) AS max_value
    FROM "{database_name}"."{table_name}"
)
SELECT * FROM percentiles;
"""

In [ ]:
wr.athena.read_sql_query(profile_numeric_sql, database=database_name)

In [13]:
profile_category_sql = f"""
WITH stats AS (
    SELECT
        'ti_cu_customer_id' AS column_name,
        COUNT(DISTINCT ti_cu_customer_id) AS unique_values,
        COUNT(*) - COUNT(ti_cu_customer_id) AS missing_values,
        CAST((SELECT ti_cu_customer_id FROM "{database_name}"."{table_name}" GROUP BY ti_cu_customer_id ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_cust_type' AS column_name,
        COUNT(DISTINCT ti_cu_cust_type) AS unique_values,
        COUNT(*) - COUNT(ti_cu_cust_type) AS missing_values,
        CAST((SELECT ti_cu_cust_type FROM "{database_name}"."{table_name}" GROUP BY ti_cu_cust_type ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_num_curr_acct' AS column_name,
        COUNT(DISTINCT ti_cu_num_curr_acct) AS unique_values,
        COUNT(*) - COUNT(ti_cu_num_curr_acct) AS missing_values,
        CAST((SELECT ti_cu_num_curr_acct FROM "{database_name}"."{table_name}" GROUP BY ti_cu_num_curr_acct ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_num_rev_acct' AS column_name,
        COUNT(DISTINCT ti_cu_num_rev_acct) AS unique_values,
        COUNT(*) - COUNT(ti_cu_num_rev_acct) AS missing_values,
        CAST((SELECT ti_cu_num_rev_acct FROM "{database_name}"."{table_name}" GROUP BY ti_cu_num_rev_acct ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_num_mtge_acct' AS column_name,
        COUNT(DISTINCT ti_cu_num_mtge_acct) AS unique_values,
        COUNT(*) - COUNT(ti_cu_num_mtge_acct) AS missing_values,
        CAST((SELECT ti_cu_num_mtge_acct FROM "{database_name}"."{table_name}" GROUP BY ti_cu_num_mtge_acct ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_num_loan_acct' AS column_name,
        COUNT(DISTINCT ti_cu_num_loan_acct) AS unique_values,
        COUNT(*) - COUNT(ti_cu_num_loan_acct) AS missing_values,
        CAST((SELECT ti_cu_num_loan_acct FROM "{database_name}"."{table_name}" GROUP BY ti_cu_num_loan_acct ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_cu_num_dep_acct' AS column_name,
        COUNT(DISTINCT ti_cu_num_dep_acct) AS unique_values,
        COUNT(*) - COUNT(ti_cu_num_dep_acct) AS missing_values,
        CAST((SELECT ti_cu_num_dep_acct FROM "{database_name}"."{table_name}" GROUP BY ti_cu_num_dep_acct ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_account_id' AS column_name,
        COUNT(DISTINCT ti_ln_account_id) AS unique_values,
        COUNT(*) - COUNT(ti_ln_account_id) AS missing_values,
        CAST((SELECT ti_ln_account_id FROM "{database_name}"."{table_name}" GROUP BY ti_ln_account_id ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_reason_closed' AS column_name,
        COUNT(DISTINCT ti_ln_reason_closed) AS unique_values,
        COUNT(*) - COUNT(ti_ln_reason_closed) AS missing_values,
        CAST((SELECT ti_ln_reason_closed FROM "{database_name}"."{table_name}" GROUP BY ti_ln_reason_closed ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_num_parties' AS column_name,
        COUNT(DISTINCT ti_ln_num_parties) AS unique_values,
        COUNT(*) - COUNT(ti_ln_num_parties) AS missing_values,
        CAST((SELECT ti_ln_num_parties FROM "{database_name}"."{table_name}" GROUP BY ti_ln_num_parties ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_account_type' AS column_name,
        COUNT(DISTINCT ti_ln_account_type) AS unique_values,
        COUNT(*) - COUNT(ti_ln_account_type) AS missing_values,
        CAST((SELECT ti_ln_account_type FROM "{database_name}"."{table_name}" GROUP BY ti_ln_account_type ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_purpose' AS column_name,
        COUNT(DISTINCT ti_ln_purpose) AS unique_values,
        COUNT(*) - COUNT(ti_ln_purpose) AS missing_values,
        CAST((SELECT ti_ln_purpose FROM "{database_name}"."{table_name}" GROUP BY ti_ln_purpose ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_original_term' AS column_name,
        COUNT(DISTINCT ti_ln_original_term) AS unique_values,
        COUNT(*) - COUNT(ti_ln_original_term) AS missing_values,
        CAST((SELECT ti_ln_original_term FROM "{database_name}"."{table_name}" GROUP BY ti_ln_original_term ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_original_loan_amount' AS column_name,
        COUNT(DISTINCT ti_ln_original_loan_amount) AS unique_values,
        COUNT(*) - COUNT(ti_ln_original_loan_amount) AS missing_values,
        CAST((SELECT ti_ln_original_loan_amount FROM "{database_name}"."{table_name}" GROUP BY ti_ln_original_loan_amount ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_payment_frequency' AS column_name,
        COUNT(DISTINCT ti_ln_payment_frequency) AS unique_values,
        COUNT(*) - COUNT(ti_ln_payment_frequency) AS missing_values,
        CAST((SELECT ti_ln_payment_frequency FROM "{database_name}"."{table_name}" GROUP BY ti_ln_payment_frequency ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_payment_method' AS column_name,
        COUNT(DISTINCT ti_ln_payment_method) AS unique_values,
        COUNT(*) - COUNT(ti_ln_payment_method) AS missing_values,
        CAST((SELECT ti_ln_payment_method FROM "{database_name}"."{table_name}" GROUP BY ti_ln_payment_method ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_val_fees' AS column_name,
        COUNT(DISTINCT ti_ln_val_fees) AS unique_values,
        COUNT(*) - COUNT(ti_ln_val_fees) AS missing_values,
        CAST((SELECT ti_ln_val_fees FROM "{database_name}"."{table_name}" GROUP BY ti_ln_val_fees ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_block_code' AS column_name,
        COUNT(DISTINCT ti_ln_block_code) AS unique_values,
        COUNT(*) - COUNT(ti_ln_block_code) AS missing_values,
        CAST((SELECT ti_ln_block_code FROM "{database_name}"."{table_name}"GROUP BY ti_ln_block_code ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_val_arrears' AS column_name,
        COUNT(DISTINCT ti_ln_val_arrears) AS unique_values,
        COUNT(*) - COUNT(ti_ln_val_arrears) AS missing_values,
        CAST((SELECT ti_ln_val_arrears FROM "{database_name}"."{table_name}" GROUP BY ti_ln_val_arrears ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
    UNION ALL
    SELECT
        'ti_ln_num_mths_in_arrears' AS column_name,
        COUNT(DISTINCT ti_ln_num_mths_in_arrears) AS unique_values,
        COUNT(*) - COUNT(ti_ln_num_mths_in_arrears) AS missing_values,
        CAST((SELECT ti_ln_num_mths_in_arrears FROM "{database_name}"."{table_name}" GROUP BY ti_ln_num_mths_in_arrears ORDER BY COUNT(*) DESC LIMIT 1) AS VARCHAR) AS most_common_value
    FROM "{database_name}"."{table_name}"
)
SELECT * FROM stats;
"""

In [ ]:
wr.athena.read_sql_query(profile_category_sql, database=database_name)

In [ ]:
# close kernel to release memory
# disregard the error message
exit(0)